# Run sandbox notebook

In [1]:
%run sandbox.ipynb

Current working directory: /home/darrenyhuang/projects/pleco-anki-server/src/resources
Loading credentials from token.json
Refreshing expired credentials
Failed to refresh credentials: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})
Authenticating with Google Drive
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=624862442234-4iconv0m99pu1luia7lub9c8m7t8of0b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A45942%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=SD38PdBTvlmJgwJMDmIIVGUktkroIy&access_type=offline


In [3]:
main()

Latest flashcard xml last modified time: 2025-01-02 12:17:32 EST-0500
1648 flashcard entries found| 1 error entries found


# Helper Methods

In [4]:
to_drop = [1030, 1118]

def drop(arr, to_drop):
    arr = arr.copy()
    for i in to_drop:
        arr.pop(i)
    return arr

In [5]:
import json

with open('flashcard_entries.json', 'r') as file:
    flashcard_entries = json.load(file)

to_del = []
for i, entry in enumerate(flashcard_entries):
    pinyin = ''.join(entry['pinyin'])
    anki_pinyin = entry['anki_pinyin']
    if pinyin != anki_pinyin:
        # print(i, entry['traditional'], pinyin, anki_pinyin)
        to_del.append(i)
for i in reversed(to_del):
    del flashcard_entries[i]
        


In [149]:
from utils.pinyin import *
from utils.html import *

In [ ]:


# Example usage
  # Path to the CC-CEDICT file
fifth_tone_pinyins = extract_fifth_tone_pinyin(file_path)
# print("Pinyin with the 5th tone:", fifth_tone_pinyins)

toneless_pinyin_set = extract_toneless_pinyin(file_path)
toneless_pinyin_trie = create_trie_from_pinyin(toneless_pinyin_set)


In [8]:
"ma" in fifth_tone_pinyins

True

In [ ]:
load_cc_cedict()
load_moedict()
get_c_variants()
load_unihan_variants()
load_manual_variants()
""

''

# label segs

In [ ]:




# # Example usage
# fullwidth_text = "ｉＯＳ 10"
# fullwidth_to_ascii(fullwidth_text)

In [13]:
chinese = "升級到 ｉＯＳ 10"
english = "ｉＯＳ 10 Upgrade to iOS 10."
print(overlap_length(chinese, english))  # Output: 6

6


In [ ]:
import re
from collections import defaultdict

    


## variants

## split chinese pinyin example sentence util

In [ ]:
import regex
import regex as re
from pypinyin import pinyin, Style








In [68]:
load_manual_pinyins.cache_clear()

## separate out bold segments for chinese and pinyin

## label segments

# test bold

In [296]:
for entry in flashcard_entries:
    if entry['traditional'] == '反正':
        segments = label_segments(entry['definition'], entry['traditional'])

# Format Entries

In [601]:
pinyin_punc_to_bold = set(["？"])

def fmt_entry(entry):
    traditional = entry.get("traditional", "")
    simplified = entry.get("simplified", "")
    pinyin = entry.get("pinyin", "")
    definition = entry.get("definition", "")
    simplified_hint = f"〔{simplified}〕" if traditional != simplified else ""

    formatted_back = ""
    formatted_back += f'<div align="left"><p><span style="font-size:32px">{traditional}{simplified_hint}</span><br/>\n'
    formatted_back += '<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span>'

    # pinyin
    for p in pinyin:
        starters = ["//", " ", "-", "→"]
        while any([p.startswith(s) for s in starters]):
            for s in starters:
                if p.startswith(s):
                    formatted_back += f'<span style="font-weight:600;">{s}</span>'
                    p = p.replace(s, "", 1)
        formatted_back += f'<span style="color:{get_pinyin_color(p)};"><span style="font-weight:600;">{p}</span></span>'
    
    # part of speech
    formatted_back += '</p>\n</div><div align="left"><p>'

    last_label = None
    for segment in label_segments(definition, traditional_word=traditional):
        if segment["label"] == "part of speech":
            if last_label == "example_sentence":
                formatted_back += '<br/>\n</p>\n</blockquote>\n<p><br/>\n'
            elif last_label == "part of speech":
                formatted_back += '<br/>\n'
            elif last_label == "english":
                formatted_back += '</p>\n<p>'
            formatted_back += f'<b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">{segment["segment"].upper().strip()}</span></span></b>'


        elif segment["label"] == "english":
            # process transition
            if last_label == "part of speech":
                formatted_back += '<br/>\n'
            # add english segmewnt
            formatted_back += segment["segment"].strip()


        elif segment["label"] == "example_sentence":
            # process transition
            if last_label == "english":
                formatted_back += '<br/>\n</p>\n'
            elif last_label == "example_sentence":
                formatted_back += '<br/>\n</p>\n</blockquote>\n'
            
            # process example sentence
            formatted_back += '<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p>'
            # chinese
            for ex_seg in segment["chinese_list_w_bold_labels"]:
                formatted_back += f'<span style="color:#0078C3;">'
                if ex_seg["bold"]:
                    formatted_back += f'<b>{ex_seg["segment"]}</b>'
                else:
                    formatted_back += ex_seg["segment"]
                formatted_back += '</span>'
            formatted_back += '<br/>\n'
            # pinyin
            for ex_seg in segment["pinyin_list_w_bold_labels"]:
                if not ex_seg["segment"]:
                    continue
                if ex_seg["bold"] or ex_seg["segment"] in pinyin_punc_to_bold:
                    formatted_back += f'<b>{ex_seg["segment"]}</b>'
                else:
                    formatted_back += f'<span style="font-weight:600;">{ex_seg["segment"]}</span>'
            formatted_back += '<br/>\n'
            # english
            formatted_back += segment["english"].strip()
            # formatted_back += '<br/>\n</p>\n</blockquote>'

        elif segment["label"] == "item_number":
            if last_label:
                formatted_back += '<br/>\n'
            if last_label == "example_sentence":
                formatted_back += '</p>\n</blockquote>\n<p>'
            formatted_back += f'<b>{segment['segment'].strip()}\t</b>'
        # elif segment["label"] == "pinyin":
        #     formatted_back += f'<span style="color:{get_pinyin_color(segment["segment"])};"><b><span style="font-size:0.80em;">{segment["segment"]} </span></b></span>'
        # else:
        #     formatted_back += f'<span style="color:#000000;"><b><span style="font-size:0.80em;">{segment["segment"]} </span></b></span>'
        else:
            print(traditional, ":", segment)
        last_label = segment["label"]

    # final
    formatted_back += '</p>'
    if last_label == "example_sentence":
        formatted_back += '\n</blockquote>'

    formatted_back += '\n</div>'

    # hack
    formatted_back = re.sub(r'VARIANT OF \uead1\ueada\d+\uead8(\p{Han}+)\uead9[\d\w]+\uead0\p{Han}+\uead2 ', r'<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">VARIANT OF </span></b></span>\1<br/>\n', formatted_back)
    formatted_back = formatted_back.replace(") a surname", ")<br/>\na surname")

    return formatted_back

In [632]:
b_entries = []
for entry in flashcard_entries:
    if ' b ' in entry['definition']:
        b_entries.append({"entry": entry, "formatted": fmt_entry(entry)})

In [634]:
len(b_entries)

5

# Grade Entries

In [226]:
class ToneColor(Enum):
    RED = "#E30000"
    GREEN = "#02B31C"
    PURPLE = "#8900BF"
    BLUE = "#1510F0"
    GREY = "#777777"  # Neutral tone


def get_pinyin_color(pinyin):
    tone_map = {
        "āēīōūǖĀĒĪŌŪǕ": ToneColor.RED,
        "áéíóúǘÁÉÍÓÚǗ": ToneColor.GREEN,
        "ǎěǐǒǔǚǍĚǏǑǓǙ": ToneColor.BLUE,
        "àèìòùǜÀÈÌÒÙǛ": ToneColor.PURPLE,
    }

    tone_color = ToneColor.GREY  # Default to neutral tone
    for chars, t in tone_map.items():
        if any(char in pinyin for char in chars):
            tone_color = t
            break

    return tone_color.value

In [261]:
from bs4 import BeautifulSoup


def reorder_bold_and_color_spans(html_text):
    # Parse the HTML text
    soup = BeautifulSoup(f"<root>{html_text}</root>", "html.parser")
    
    # Find all bold tags
    bold_tags = soup.find_all("b")
    
    for bold_tag in bold_tags:
        # Check if the bold tag contains a span with the specific color #0078C3
        color_span = bold_tag.find("span", style=lambda s: s and "color:#0078C3" in s)
        
        if color_span:
            # Get the content of the span
            content = color_span.contents
            
            # Create new structure: span outside, b inside
            new_span = soup.new_tag("span")
            new_span["style"] = color_span["style"]
            
            new_bold = soup.new_tag("b")
            new_bold.extend(content)
            
            new_span.append(new_bold)
            
            # Replace the old structure with the new one
            bold_tag.replace_with(new_span)
    
    # Convert the soup back to a string
    return str(soup).replace("<root>", "").replace("</root>", "")

In [400]:
def fix_separated_pos_tags(html_text):
    pos_text = '<span style="font-size:0.80em;"><span style="color:#B4B4B4;">.*?</span></span>'
    patt = f'({pos_text})</b> <b>({pos_text})'
    previous_text = ""
    current_text = html_text
    while current_text != previous_text:
        previous_text = current_text
        current_text = re.sub(patt, r'\1</b><br/>\n<b>\2', current_text)
    return current_text

In [639]:

def grade_fmt_entry(flashcard_entries, n_error_char_show=10, to_drop=[], stop_at_fail=False, print_at_fail=False):
    flashcard_entries = drop(flashcard_entries, to_drop)
    correct_count = 0
    length_diff_count = 0
    wrong_count = 0
    for i, entry in enumerate(flashcard_entries):
        expected = entry['formatted_back']
        expected = expected.replace(' ;=""', ";")
        expected = reorder_nested_spans(expected)
        expected = reorder_bold_and_color_spans(expected)
        expected = re.sub(r"<plecoentry.*?</plecoentry>$", "", expected)
        expected = expected.replace("\xa0", " ")
        expected = fix_separated_pos_tags(expected)
        result = fmt_entry(entry)
        result = re.sub(r" See \uead1\ueada\d+\uead8\p{Han}+\uead9[\w\d]+\uead0\p{Han}+\uead2", "", result)

        # result = re.sub(r"[()]", "", result)
        # expected = re.sub(r"[()]", "", expected)
        
        if expected != result and re.sub(r"[()]", "", expected) != re.sub(r"[()]", "", result):
            for j in range(min(len(expected), len(result))):
                if expected[j] != result[j]:
                    wrong_count += 1
                    if stop_at_fail or print_at_fail:
                        # print(f"Total correct entries: {correct_count}")
                        # print(f"Total wrong entries: {wrong_count}")
                        # print(f"Total entries differing only in length: {length_diff_count}")
                        # print(f"Entry {i} differs at character {j}:")
                        print("{i} wrd:", repr(entry['traditional']))
                        print(f"Exp: {repr(expected[j:j+n_error_char_show])}...")
                        print(f"Got: {repr(result[j:j+n_error_char_show])}...")
                        print(f"Def: {entry['definition']}...")
                        # print("exp:", repr(expected))
                        # print("res:", repr(result))
                        if stop_at_fail:
                            return
                    break
            else:
                length_diff_count += 1
                j = len(result)
                if stop_at_fail:
                    print(f"Total correct entries: {correct_count}")
                    print(f"Total wrong entries: {wrong_count}")
                    print(f"Total entries differing only in length: {length_diff_count}")
                    print(f"{i}: {repr(expected[j:j+n_error_char_show])}...")
                    print("wrd:", repr(entry['traditional']))
                    print("def:", repr(entry['definition']))
                    print("exp:", repr(expected))
                    print("res:", repr(result))
                    return
                continue
        else:
            correct_count += 1
    print(f"Total correct entries: {correct_count}/{correct_count + wrong_count}")
    print(f"Total wrong entries: {wrong_count}/{correct_count + wrong_count}")
    print(f"Total entries differing only in length: {length_diff_count}")

## actual grade run

In [631]:
word = '所'
entry = [entry for entry in flashcard_entries if entry['traditional'] == word][0]
label_segments(entry['definition'], traditional_word=word)


[{'segment': 'noun ', 'label': 'part of speech'},
 {'segment': '1', 'label': 'item_number'},
 {'segment': 'place ', 'label': 'english'},
 {'label': 'example_sentence',
  'chinese': '住所 ',
  'pinyin': 'zhùsuǒ ',
  'english': 'dwelling place; residence ',
  'chinese_list': ['住', '所'],
  'pinyin_list': ['zhù', 'suǒ'],
  'ignored_pinyin': '',
  'chinese_list_w_bold_labels': [{'segment': '住', 'bold': False},
   {'segment': '所', 'bold': True}],
  'pinyin_list_w_bold_labels': [{'segment': 'zhù', 'bold': False},
   {'segment': 'suǒ', 'bold': True}]},
 {'segment': '2', 'label': 'item_number'},
 {'segment': 'office; bureau; institute ', 'label': 'english'},
 {'label': 'example_sentence',
  'chinese': '指揮所 ',
  'pinyin': 'zhǐhuīsuǒ ',
  'english': 'command post ',
  'chinese_list': ['指', '揮', '所'],
  'pinyin_list': ['zhǐ', 'huī', 'suǒ'],
  'ignored_pinyin': '',
  'chinese_list_w_bold_labels': [{'segment': '指揮', 'bold': False},
   {'segment': '所', 'bold': True}],
  'pinyin_list_w_bold_labels': [{'

In [630]:
word = '所'
entry = [entry for entry in flashcard_entries if entry['traditional'] == word][0]
grade_fmt_entry([entry], 1000, to_drop=[], stop_at_fail=True)

Total correct entries: 0
Total wrong entries: 1
Total entries differing only in length: 0
Entry 0 differs at character 2769:
Exp: '<br/>\n<b>a</b> [with 的 to modify a noun]<br/>\n</p>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">我</span><span style="color:#0078C3;"><b>所</b></span><span style="color:#0078C3;">認識的人</span><br/>\n<span style="font-weight:600;">wǒ </span><b>suǒ</b><span style="font-weight:600;"> rènshi de rén</span><br/>\nthe people I know<br/>\n</p>\n</blockquote>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">大家</span><span style="color:#0078C3;"><b>所</b></span><span style="color:#0078C3;">提的意見</span><br/>\n<span style="font-weight:600;">dàjiā </span><b>suǒ</b><span style="font-weight:600;"> tí de yìjian</span><br/>\nthe opinions various people put f

In [603]:
grade_fmt_entry(flashcard_entries, 1000, to_drop=[])

令: Error processing example sentence: {'label': 'example_sentence', 'chinese': '(散曲) ', 'pinyin': 's', 'english': 'song (i.e. not in a sequence, corresponding to one aria in the drama) 1 [usu. in tune titles] ', 'chinese_list': ['(散曲)'], 'pinyin_list': [''], 'ignored_pinyin': ''}, converting to english
Total correct entries: 1318/1604
Total wrong entries: 286/1604
Total entries differing only in length: 0


In [640]:
grade_fmt_entry(flashcard_entries, 1000, to_drop=[14, 21, 29, 38, 41, 43, 45, 48, 53, 61, 63, 65, 76, 78, 80, 82] + [84, 85, 85, 89, 116, 135, 139, 143, 160, 162, 162], print_at_fail=True)
# grade_fmt_entry(flashcard_entries[6:], 1000)

{i} wrd: '滾開'
Exp: 'br/>\n<b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">VERB</span></span></b><br/>\n<b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">VULGAR</span></span></b><br/>\nbeat it; scram<br/>\n</p>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;"><b>滾開</b></span><span style="color:#0078C3;">, 要不我揍你！</span><br/>\n<b>Gǔnkāi</b><span style="font-weight:600;">, yào bù wǒ zòu nǐ!</span><br/>\nGet out, or I’ll spank you!<br/>\n</p>\n</blockquote>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">這兒沒你的事兒, </span><span style="color:#0078C3;"><b>滾開</b></span><span style="color:#0078C3;">！</span><br/>\n<span style="font-weight:600;">Zhèr méi nǐ de shìr, </span><b>gǔnkāi</b><span style="font-weight:600;">!</span><br/>\nNone of your business

In [162]:
fmt_entry(flashcard_entries[0])

'<div align="left"><p><span style="font-size:32px">艘</span><br/>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#E29999;"><span style="font-weight:600;">sōu</span></span></p>\n</div><div align="left"><p>'

# TODOS!
include removed () from the example sentence into the final example sentence output